In [18]:

import tensorflow as tf
import numpy as np
import re,os,pathlib
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

In [19]:
print(tf.__version__)

2.3.0


In [20]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [21]:
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
             'X', 'Y', 'Z', 'O']
#‘I’是手动加的

ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
       'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O','I']

In [22]:
pinyin2char = {'Anhui':'皖',
 'Beijing':'京',
 'Chongqing':'渝',
 'Fujian':'闽',
 'Gansu':'甘',
 'Guangdong':'粤',
 'Guangxi':'桂',
 'Guizhou':'贵',
 'Hainan':'琼',
 'Hebei':'冀',
 'Heilongjiang':'黑',
 'Henan':'豫',
 'Hubei':'鄂',
 'Hunan':'湘',
 'InnerMongolia':'蒙',
 'Jiangsu':'苏',
 'Jiangxi':'赣',
 'Jilin':'吉',
 'Liaoning':'辽',
 'Ningxia':'宁',
 'Qinghai':'青',
 'Shaanxi':'陕',
 'Shandong':'鲁',
 'Shanghai':'沪',
 'Shanxi':'晋',
 'Sichuan':'川',
 'Tianjin':'津',
 'Tibet':'藏',
 'Xinjiang':'新',
 'Yunnan':'云',
 'Zhejiang':'浙',
 'police':'警'}

In [23]:
dataset_path = r'C:\Users\Raymond\PycharmProjects\tensorflow2.0\datasets'

In [24]:
all_provinces = []
all_provinces+=provinces
# all_chars+=alphabets
# all_chars+=ads
all_provinces = list(set(all_provinces))
all_provinces.sort()

# all_chars.insert(69,'[sep]')

provinces2int = {}
for i,key in enumerate(all_provinces):
    provinces2int[key] = i

In [25]:
len(provinces2int)

34

In [26]:
all_chars = []
# all_chars+=provinces
all_chars+=alphabets
all_chars+=ads
all_chars = list(set(all_chars))
all_chars.sort()

all_chars.insert(37,'[sep]')

char2int = {}
for i,key in enumerate(all_chars):
    char2int[key] = i

In [27]:
len(char2int)

37

In [28]:
char2int

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'A': 10,
 'B': 11,
 'C': 12,
 'D': 13,
 'E': 14,
 'F': 15,
 'G': 16,
 'H': 17,
 'I': 18,
 'J': 19,
 'K': 20,
 'L': 21,
 'M': 22,
 'N': 23,
 'O': 24,
 'P': 25,
 'Q': 26,
 'R': 27,
 'S': 28,
 'T': 29,
 'U': 30,
 'V': 31,
 'W': 32,
 'X': 33,
 'Y': 34,
 'Z': 35,
 '[sep]': 36}

In [29]:
provinces2int.keys()

dict_keys(['O', '云', '京', '冀', '吉', '学', '宁', '川', '新', '晋', '桂', '沪', '津', '浙', '渝', '湘', '琼', '甘', '皖', '粤', '苏', '蒙', '藏', '警', '豫', '贵', '赣', '辽', '鄂', '闽', '陕', '青', '鲁', '黑'])

In [30]:
char2int.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[sep]'])

In [31]:
charlist = list(char2int.keys())

In [32]:
# lll=[]
def get_synthetic_np():
    data_root = os.path.join(dataset_path,'Synthetic_Chinese_License_Plates')
    data_root = pathlib.Path(data_root)
    print(data_root)
    annotation = os.path.join(data_root,'annotation')
    with open(annotation) as f:
        datas = f.readlines()
    p = re.compile("[<|>]")
    data_gen = {}
    for d in datas:
        path,plate = d.split()
        path = os.path.join(data_root,path)
        path = pathlib.Path(path)
        split_plate = p.split(plate)
        wj,province,num = split_plate[0:3]
        try:
            police = split_plate[3]
            police = pinyin2char[police]
        except:
            police = ''
        if wj or police:
            continue
        province_label = pinyin2char[province]
        province_label = provinces2int[province_label]
#         str_plate = ''.join([wj,pinyin2char[province],num,police])
        char_label = [char2int[i] for i in num]
#         if len(label) == 6:
#             print(str_plate)
#         lll.append(len(label))
        paddings_len = 7 - len(char_label)
        for i in range(paddings_len):
            char_label.append(36)
#         province_label = np.asarray(province_label)
        char_label = np.asarray(char_label)
        
        cor = np.asarray([0,0,94,24])
        cor = np.expand_dims(cor,axis=0)
        try:
            data_gen[province_label].append([str(path),province_label,char_label,cor])
        except:
            data_gen[province_label] = []
            data_gen[province_label].append([str(path),province_label,char_label,cor])

    res_data_gen = []
    for i in data_gen.keys():
        res_pos_features = np.asarray(data_gen[i])
        res_data_gen.append(res_pos_features)

    data_np = np.concatenate(res_data_gen)
    return data_np

def get_gen_np():
    #生成车牌数据集
    p2 = re.compile("[_|.]")
    data_root_gen = os.path.join(dataset_path,'small_new_energy')
    data_root_gen = pathlib.Path(data_root_gen)
    print(data_root_gen)

    # 生成车牌数据集
    data_gen = {}
    for file in data_root_gen.glob('*.jpg'):
        # 转换为车牌字符串
        str_plate = p2.split(file.parts[-1])[1]
        province_label = provinces2int[str_plate[0]]
        char_label = [char2int[i] for i in str_plate[1:]]
#         label = [char2int[i] for i in str_plate[1]]
        if file.parts[-2] == 'small_new_energy':
            width = 246
        else:
            width = 226
            char_label.append(36)
        char_label = np.asarray(char_label)
        
        cor = np.asarray([0,0,width,72])
        cor = np.expand_dims(cor,axis=0)

        

        try:
#             data_gen[label[0]].append([str(file),label,cor])
            data_gen[province_label].append([str(file),province_label,char_label,cor])
        except:
            data_gen[province_label] = []
#             data_gen[label[0]].append([str(file),label,cor])
            data_gen[province_label].append([str(file),province_label,char_label,cor])

    res_data_gen = []
    for i in data_gen.keys():
        res_pos_features = np.asarray(data_gen[i])
        res_data_gen.append(res_pos_features)


    data_np = np.concatenate(res_data_gen)
    return data_np

def get_ccpd_np():
    p = re.compile("[_|&]")
    data_root = os.path.join(dataset_path,'CCPD2019')
    data_root = pathlib.Path(data_root)
    print(data_root)

    #resample的数据集
    data = {}
    for file in data_root.glob('*/*.jpg'):
        if file.parts[-2] not in ['ccpd_base','ccpd_green']:
            continue
        try:
            cor = p.split(file.parts[-1].split('-')[2])
        except:
            continue
        cor = [int(i) for i in cor]
        cor[0],cor[1] = max(0,cor[0]-random.randint(0,30)),max(0,cor[1]-random.randint(0,30))
        cor[2],cor[3] = min(720,cor[2]+random.randint(0,30)),min(1160,cor[3]+random.randint(0,30))
        cor = np.asarray([cor[0],cor[1],cor[2],cor[3]])
        cor = np.expand_dims(cor,axis=0)

        str_plate = file.parts[-1].split('-')[-3].split('_')
        int_plate = [int(i) for i in str_plate]
        # 转换为车牌字符串
        str_plate = [provinces[int_plate[0]],alphabets[int_plate[1]]] + [ads[i] for i in int_plate[2:]]
#         label = [char2int[i] for i in str_plate]
        province_label = provinces2int[str_plate[0]]
        char_label = [char2int[i] for i in str_plate[1:]]
        if len(char_label) == 6:
            char_label.append(36)
        char_label = np.asarray(char_label)

        try:
#             data[province_label].append([str(file),label,cor])
            data[province_label].append([str(file),province_label,char_label,cor])
        except:
            data[province_label] = []
#             data[label[0]].append([str(file),label,cor])
            data[province_label].append([str(file),province_label,char_label,cor])

    res_data = []
    for i in data.keys():
        ids = np.arange(len(data[i]))
        choices = np.random.choice(ids, 1000)

        res_pos_features = np.asarray(data[i])
        res_data.append(res_pos_features)


    data_np = np.concatenate(res_data)
    return data_np

gen_np = get_gen_np()
ccpd_np = get_ccpd_np()
synthetic_np = get_synthetic_np()
synthetic_np,_ = train_test_split(synthetic_np,test_size=0.8, shuffle=True)

data_np = np.concatenate([gen_np,ccpd_np,synthetic_np])
# np.random.shuffle(data_np)
train_np,test_np = train_test_split(data_np,test_size=0.33, shuffle=True)
print(data_np.shape,train_np.shape,test_np.shape)



C:\Users\Raymond\PycharmProjects\tensorflow2.0\datasets\small_new_energy


C:\Users\Raymond\Anaconda3\envs\tf23\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


C:\Users\Raymond\PycharmProjects\tensorflow2.0\datasets\CCPD2019
C:\Users\Raymond\PycharmProjects\tensorflow2.0\datasets\Synthetic_Chinese_License_Plates
(278086, 4) (186317, 4) (91769, 4)


In [33]:
ccpd_np.shape

(211769, 4)

In [34]:
data_np[0]

array(['C:\\Users\\Raymond\\PycharmProjects\\tensorflow2.0\\datasets\\small_new_energy\\000000450_宁MFJ2835.jpg',
       6, array([22, 15, 19,  2,  8,  3,  5]),
       array([[  0,   0, 246,  72]])], dtype=object)

In [35]:
# img = r'C:\Users\Raymond\PycharmProjects\tensorflow2.0\datasets\CCPD2019\ccpd_green\0183796296296-90_95-345&874_742&979-734&979_345&977_346&874_742&876-0_0_3_24_27_29_31_28-90-324.jpg'
# img = pathlib.Path(img)
# p = re.compile("[_|&]")
# cor = p.split(img.parts[-1].split('-')[2])
# cor = [int(i) for i in cor]
# cor = np.asarray([cor[0],cor[1],cor[2],cor[3]])
# img = tf.io.read_file(str(img))
# img = tf.io.decode_image(img)
# print(img.shape,cor,cor[3]-cor[1],cor[2]-cor[0])
# tf.image.crop_to_bounding_box(
#         img, cor[1], cor[0], cor[3]-cor[1], cor[2]-cor[0]
#         )
# # tf.image.crop_to_bounding_box(
# #         img, cor[1], cor[0], cor[3]-cor[1], cor[2]-cor[0]
# #         )

In [36]:
def train_gen():
    for file,province_label,char_label,cor in train_np:
        yield str(file),province_label,char_label,cor
def val_gen():
    for file,province_label,char_label,cor in test_np:
        yield str(file),province_label,char_label,cor

train_ds = tf.data.Dataset.from_generator(train_gen,(tf.string,tf.int32,tf.int32,tf.int32))
val_ds = tf.data.Dataset.from_generator(val_gen,(tf.string,tf.int32,tf.int32,tf.int32))

In [37]:
# set(lll)

In [38]:
gen_np.shape,ccpd_np.shape,synthetic_np.shape,data_np.shape

((12600, 4), (211769, 4), (53717, 4), (278086, 4))

In [39]:
VAL_SIZE = 100000
REPEAT = 6
DATA_SIZE = train_np.shape[0]
TRAIN_SIZE = (DATA_SIZE- VAL_SIZE) * REPEAT

In [40]:
len(char2int)

37

In [41]:
train_ds = train_ds.cache("train_cache")
val_ds = val_ds.cache("val_cache")

In [42]:
if not tf.io.gfile.exists('train_cache.index'):
    for _ in tqdm(train_ds.as_numpy_iterator()):
        pass
if not tf.io.gfile.exists('val_cache.index'):
    for _ in tqdm(val_ds.as_numpy_iterator()):
        pass

In [43]:
# for p,_,_ in train_ds.as_numpy_iterator():
#     try:
#         img = tf.io.read_file(p)
#         img = tf.io.decode_image(img)
#     except:
#         print(p)

In [44]:
def postprocess(path,province_label,char_label,cor):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img)
#     img = tf.image.rgb_to_grayscale(img)
    img = tf.image.convert_image_dtype(img,tf.float32)
    cor = cor[0]
    img = tf.image.crop_to_bounding_box(
        img, cor[1], cor[0], cor[3]-cor[1], cor[2]-cor[0]
        )
#     img = tf.image.resize(img,[65,210])
#     img = tf.image.random_crop(img,[60,200,3])
    img = tf.image.resize(img,[64,224])
#     img = tf.transpose(img, perm=[1, 0, 2])

    return img,(province_label,char_label)


In [45]:
next(train_ds.as_numpy_iterator())

(b'C:\\Users\\Raymond\\PycharmProjects\\tensorflow2.0\\datasets\\CCPD2019\\ccpd_base\\0316810344828-89_95-162&600_466&700-492&690_158&715_164&605_498&580-0_0_10_25_23_29_32-133-45.jpg',
 18,
 array([10, 21,  1, 35,  5,  8, 36]),
 array([[139, 598, 494, 726]]))

In [46]:
# for i,l,c in tqdm(train_ds.as_numpy_iterator()):
#     img = tf.io.read_file(i)
#     img = tf.io.decode_image(img)
# #     tf.print(img.shape)
#     cor = c[0]
#     h,w,_ = img.shape
#     try:
#         img = tf.image.crop_to_bounding_box(
#         img, cor[1], cor[0], cor[3]-cor[1], cor[2]-cor[0]
#         )
#     except:
#         print(i)
# #     if h<(c[3]-c[1]) or w<c[2]-c[0]:
# #         print(i)

In [47]:
train_ds = train_ds.shuffle(1000,reshuffle_each_iteration=True).prefetch(tf.data.experimental.AUTOTUNE)
train_ds = train_ds.map(postprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_ds = val_ds.map(postprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [48]:
# for _ in tqdm(train_ds.as_numpy_iterator()):
#     pass

In [49]:
# def postprocess(img,label,cor):
#     img = tf.io.read_file(path)
#     img = tf.io.decode_image(img)
#     img = tf.image.convert_image_dtype(img,tf.float32)
#     cor = cor[0]
#     img = tf.image.crop_to_bounding_box(
#         img, cor[1], cor[0], cor[3]-cor[1], cor[2]-cor[0]
#         )
#     img = tf.image.resize(img,[65,210])
#     img = tf.image.random_crop(img,[60,200,3])
#     img = tf.transpose(img, perm=[1, 0, 2])

#     return img,label

# def map_fn(img, label,cor):
#   # py_func doesn't set the shape of the returned tensors.
#   img,label = tf.py_function(postprocess, 
#                                        inp=[img, label,cor], 
#                                        Tout=(tf.float32, tf.int32))

#   # `tf.data.Datasets` work best if all components have a shape set
#   #  so set the shapes manually: 
#   img.set_shape([200,60,3])
#   label.set_shape([None])

#   return img,label


# train_ds = train_ds.map(map_fn,num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat(3)
# val_ds = val_ds.map(map_fn,num_parallel_calls=tf.data.experimental.AUTOTUNE)
# test_ds = test_ds.map(map_fn,num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [50]:
BATCH_SIZE = 32

In [51]:
train_ds = train_ds.batch(BATCH_SIZE)
val_ds = val_ds.batch(BATCH_SIZE)

In [52]:
# image_test,_,_ = postprocess(p,l,c)
# image_test = tf.image.resize(image_test,[30,100])
# # print(image_test)

# plt.imshow(image_test)
# plt.show()

In [53]:
tests,labels = next(train_ds.unbatch().shuffle(30).as_numpy_iterator())
plt.imshow(tests)
plt.show()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 222: invalid continuation byte

In [ ]:
img_tests,labels = next(train_ds.as_numpy_iterator())

In [ ]:
labels[1].shape

In [ ]:
img_tests.shape

In [54]:
def dwDarknetConv(x, filters, size, strides=1, batch_norm=True,bias_initializer=tf.keras.initializers.Zeros()):
    if strides == 1:
        padding = 'same'
    else:
        x = ZeroPadding2D(((1, 0), (1, 0)))(x)  # top left half-padding
        padding = 'valid'
    x = tf.keras.layers.SeparableConv2D(filters=filters, kernel_size=size,
               strides=strides, padding=padding,
#                kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01),
#                bias_initializer=bias_initializer,
               use_bias=not batch_norm, kernel_regularizer=l2(0.0005))(x)
    if batch_norm:
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    return x

In [55]:
from tensorflow.keras.layers import (
    Add,
    Concatenate,
    Conv2D,
    Input,
    Lambda,
    LeakyReLU,
    MaxPool2D,
    UpSampling2D,
    ZeroPadding2D,
)
from tensorflow.keras.regularizers import l2

In [56]:
def dwYoloConv(filters, name=None):
    def yolo_conv(x_in):
        if isinstance(x_in, tuple):
            inputs = Input(x_in[0].shape[1:]), Input(x_in[1].shape[1:])
            x, x_skip = inputs

            # concat with skip connection
            x = dwDarknetConv(x, filters, 1)
            x = UpSampling2D(2)(x)
            x = Concatenate()([x, x_skip])
        else:
            x = inputs = Input(x_in.shape[1:])

        x = dwDarknetConv(x, filters, 1)
        x = dwDarknetConv(x, filters * 2, 3)
        x = dwDarknetConv(x, filters, 1)
#         x = dwDarknetConv(x, filters * 2, 3)
#         x = dwDarknetConv(x, filters, 1)
        return tf.keras.Model(inputs, x, name=name)(x_in)
    return yolo_conv

def dwDarknetTiny(name=None):
    x = inputs = Input([None, None, 3])
    x = dwDarknetConv(x, 16, 3)
    x = MaxPool2D(2, 2, 'same')(x)
    x = dwDarknetConv(x, 32, 3)
    x = MaxPool2D(2, 2, 'same')(x)
    x = dwDarknetConv(x, 64, 3)
    x = MaxPool2D(2, 2, 'same')(x)
    x = dwDarknetConv(x, 128, 3)
    x = MaxPool2D(2, 2, 'same')(x)
    x = x_8 = dwDarknetConv(x, 256, 3)  # skip connection
    x = MaxPool2D(2, 2, 'same')(x)
    x = dwDarknetConv(x, 512, 3)
    x = MaxPool2D(2, 1, 'same')(x)
    x = dwDarknetConv(x, 1024, 3)
    return tf.keras.Model(inputs, (x_8, x), name=name)

def dwYoloV3Tiny(size=None):
    x = inputs = Input([64, 224, 3])
    x = tf.keras.layers.Lambda(lambda x: tf.transpose(x,[0,2,1,3]))(x)

    x_8, x = dwDarknetTiny(name='yolo_darknet')(x)
    output_0 = tf.keras.layers.Flatten()(x)
    output_0 = tf.keras.layers.Dense(34)(output_0)
    output_0 = tf.keras.layers.Activation('softmax')(output_0)

    x = dwYoloConv(256, name='yolo_conv_0')(x)
    x = dwYoloConv(128, name='yolo_conv_1')((x, x_8))
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(x)
    output_1 = tf.keras.layers.Conv1D(128,3)(x)
    output_1 = tf.keras.layers.LeakyReLU(alpha=0.1)(output_1)
    output_1 = tf.keras.layers.Dense(37)(output_1)
    output_1 = tf.keras.layers.Activation('softmax')(output_1)

    return tf.keras.Model(inputs, (output_0,output_1), name='yolov3_tiny')

In [57]:
# loss: 0.3186 - val_loss: 0.3178 - acc: 0.92 - Total params: 131,744
# def dwDarknetConv(x, filters, size, strides=1, batch_norm=True,bias_initializer=tf.keras.initializers.Zeros()):
#     if strides == 1:
#         padding = 'same'
#     else:
#         x = ZeroPadding2D(((1, 0), (1, 0)))(x)  # top left half-padding
#         padding = 'valid'
#     x = tf.keras.layers.SeparableConv2D(filters=filters, kernel_size=size,
#                strides=strides, padding=padding,
# #                kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01),
# #                bias_initializer=bias_initializer,
#                use_bias=not batch_norm, kernel_regularizer=l2(0.0005))(x)
#     if batch_norm:
#         x = tf.keras.layers.BatchNormalization()(x)
#         x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
#     return x

def conv1d_model():
    inp = tf.keras.Input(shape=(64,224,3),dtype=tf.float32,name="input")
    x = tf.keras.layers.Lambda(lambda x: tf.transpose(x,[0,2,1,3]))(inp)
    features_1 = dwDarknetConv(x,32,3,strides=2)
    
    features_2 = dwDarknetConv(features_1,32,3)

    for _ in range(1):
        features_2 = dwDarknetConv(features_2,32,3,strides=2)

    for _ in range(1):
        features_2 = dwDarknetConv(features_2,64,3)

    features_3 = dwDarknetConv(features_2,64,3,strides=2)

    for _ in range(1):
        features_3 = dwDarknetConv(features_3,64,3)
  
    features_4 = dwDarknetConv(features_3,64,3,strides=1)

    for _ in range(1):
        features_4 = dwDarknetConv(features_4,64,3)
#     out = tf.keras.layers.concatenate([features_4,features_3,features_2], axis=-1)
    out = dwDarknetConv(features_4,128,3,strides=2)
    output_0 = dwDarknetConv(out,128,3,strides=2)
    output_0 = tf.keras.layers.Flatten()(output_0)
    output_0 = tf.keras.layers.Dense(34,kernel_regularizer=l2(0.0005))(output_0)
    output_0 = tf.keras.layers.Activation('softmax',name='output_0')(output_0)
    
    output_1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(out)
    output_1 = tf.keras.layers.ZeroPadding1D(1)(output_1)
    output_1 = tf.keras.layers.Conv1D(128,3,activation='relu')(output_1)
    
    output_1 = tf.keras.layers.ZeroPadding1D(1)(output_1)
    output_1 = tf.keras.layers.Conv1D(128,3,activation='relu')(output_1)
    output_1 = tf.keras.layers.ZeroPadding1D(1)(output_1)
    output_1 = tf.keras.layers.Conv1D(128,3,activation='relu')(output_1)
    
    output_1 = tf.keras.layers.ZeroPadding1D(1)(output_1)
    output_1 = tf.keras.layers.Conv1D(64,3)(output_1)
    output_1 = tf.keras.layers.Dense(37)(output_1)
    output_1 = tf.keras.layers.Activation('softmax',name='output_1')(output_1)

    return tf.keras.Model(inp, (output_0,output_1), name='cnn_ocr')

In [58]:
# def conv1d_model():
#     inp = tf.keras.Input(shape=(64,224,3),dtype=tf.float32,name="input")
#     x = tf.keras.layers.Lambda(lambda x: tf.transpose(x,[0,2,1,3]))(inp)
#     features_1 = dwDarknetConv(x,32,3,strides=2)
    
#     features_2 = dwDarknetConv(features_1,32,3)

#     for _ in range(1):
#         features_2 = dwDarknetConv(features_2,32,3,strides=2)

#     for _ in range(1):
#         features_2 = dwDarknetConv(features_2,64,3)

#     features_3 = dwDarknetConv(features_2,64,3,strides=2)

#     for _ in range(1):
#         features_3 = dwDarknetConv(features_3,64,3)
  
#     features_4 = dwDarknetConv(features_3,64,3,strides=1)

#     for _ in range(1):
#         features_4 = dwDarknetConv(features_4,64,3)
# #     out = tf.keras.layers.concatenate([features_4,features_3,features_2], axis=-1)
#     out = dwDarknetConv(features_4,128,3,strides=2)
# #     output_0 = dwDarknetConv(out,64,3,strides=2)
# #     output_0 = tf.keras.layers.Flatten()(output_0)
# #     output_0 = tf.keras.layers.Dense(34)(output_0)
# #     output_0 = tf.keras.layers.Activation('softmax',name='output_0')(output_0)
    
#     out = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(out)
#     out = tf.keras.layers.ZeroPadding1D(1)(out)
#     out = tf.keras.layers.Conv1D(128,3,activation='relu')(out)
    
#     out = tf.keras.layers.ZeroPadding1D(1)(out)
#     out = tf.keras.layers.Conv1D(128,3,activation='relu')(out)
#     out = tf.keras.layers.ZeroPadding1D(1)(out)
#     out = tf.keras.layers.Conv1D(128,3,activation='relu')(out)
    
#     out = tf.keras.layers.ZeroPadding1D(1)(out)
#     out = tf.keras.layers.Conv1D(64,3)(out)
    
#     output_0 = tf.keras.layers.Flatten()(out)
#     output_0 = tf.keras.layers.Dense(34)(output_0)
#     output_0 = tf.keras.layers.Activation('softmax',name='output_0')(output_0)
    
#     output_1 = tf.keras.layers.Dense(37)(out)
#     output_1 = tf.keras.layers.Activation('softmax',name='output_1')(output_1)

#     return tf.keras.Model(inp, (output_0,output_1), name='cnn_ocr')

In [59]:
def crnn_model(filter_size = 32):
    inp = tf.keras.Input(shape=(64,224,3),dtype=tf.float32,name="input")
    x = tf.keras.layers.Lambda(lambda x: tf.transpose(x,[0,2,1,3]))(inp)
    features_1 = dwDarknetConv(x,filter_size,3,strides=2)
    
    features_2 = dwDarknetConv(features_1,filter_size,3)

    for _ in range(1):
        features_2 = dwDarknetConv(features_2,filter_size,3,strides=2)

    for _ in range(1):
        features_2 = dwDarknetConv(features_2,filter_size*2,3)

    features_3 = dwDarknetConv(features_2,filter_size*2,3,strides=2)

    for _ in range(1):
        features_3 = dwDarknetConv(features_3,filter_size*2,3)
  
    features_4 = dwDarknetConv(features_3,filter_size*2,3,strides=1)

    for _ in range(1):
        features_4 = dwDarknetConv(features_4,filter_size*2,3)
#     out = tf.keras.layers.concatenate([features_4,features_3,features_2], axis=-1)
    out = dwDarknetConv(features_4,filter_size*4,3,strides=2)
    output_0 = dwDarknetConv(out,filter_size*2,3,strides=2)
    output_0 = tf.keras.layers.Flatten()(output_0)
    output_0 = tf.keras.layers.Dense(34)(output_0)
    output_0 = tf.keras.layers.Activation('softmax',name='output_0')(output_0)
    
    output_1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(out)
    output_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(filter_size*4, return_sequences=True,stateful=False,dropout=0.1))(output_1)
    output_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(filter_size*2, return_sequences=True,stateful=False,dropout=0.1))(output_1)
    output_1 = tf.keras.layers.Dense(37,activation='softmax')(output_1)

    return tf.keras.Model(inp, (output_0,output_1), name='cnn_ocr')

In [60]:
# cnn_model = dwYoloV3Tiny()
cnn_model = conv1d_model()
# cnn_model = crnn_model(32)

In [ ]:
# inp = tf.keras.Input(shape=(200,60,3),dtype=tf.float32,name="input")
# out = dwDarknetConv(inp, 32, 3)
# # out = tf.keras.layers.SeparableConv2D(32, (3, 3), activation='relu', padding="same", kernel_initializer="he_normal")(inp)
# # out = tf.keras.layers.BatchNormalization()(out)
# out = tf.keras.layers.MaxPooling2D((2, 2))(out)
# out = dwDarknetConv(out, 64, 3)
# # out = tf.keras.layers.SeparableConv2D(64, (3, 3), activation='relu', padding="same", kernel_initializer="he_normal")(out)
# # out = tf.keras.layers.BatchNormalization()(out)
# out = tf.keras.layers.MaxPooling2D((2, 2))(out)
# out = dwDarknetConv(out, 128, 3,batch_norm=False)
# out = dwDarknetConv(out, 64, 3,batch_norm=False)
# out = tf.keras.layers.Conv2D(64,(1,1))(out)
# out = tf.keras.layers.BatchNormalization()(out)
# out = tf.keras.layers.LeakyReLU(alpha=0.1)(out)
# out = tf.keras.layers.Flatten()(out)

# # out = tf.keras.layers.SeparableConv2D(256, (3, 3), activation='relu')(out)
# # out = tf.keras.layers.BatchNormalization()(out)

# # out = tf.keras.layers.Lambda(lambda x: tf.transpose(x,[0,2,1,3]))(out)

# # out = tf.keras.layers.Dense(15*70)(out)
# # out = tf.keras.layers.LeakyReLU(alpha=0.1)(out)
# out = tf.keras.layers.Reshape(( 15, -1))(out)
# out = tf.keras.layers.Dense(70)(out)
# out = tf.keras.layers.Activation('softmax')(out)
# # out = tf.keras.layers.Dropout(0.2)(out)
# cnn_model = tf.keras.Model(inp,out)

In [ ]:
cnn_model.outputs

In [ ]:
cnn_out = cnn_model(img_tests)

In [ ]:
rnn_out = cnn_out

In [ ]:
cnn_out[0].shape

In [61]:
model = cnn_model

In [ ]:
model.summary()


In [ ]:
labels[0].shape,labels[1].shape

In [ ]:
labels

In [ ]:
np.ones([BATCH_SIZE],dtype="int32") * 7

In [ ]:
#接收logits
def loss(y_true, y_pred):
    batch_size = tf.cast(tf.shape(y_pred)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")
    input_length = input_length * tf.ones(shape=(batch_size), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_size), dtype="int64")
    y_pred = tf.math.log(y_pred + tf.keras.backend.epsilon())
    loss_ = tf.nn.ctc_loss(labels=y_true,
                           logits=y_pred,
                           label_length=label_length,
                           logit_length=input_length,
                           logits_time_major = False)
    return tf.expand_dims(loss_,axis=-1)


In [ ]:
#接收softmax
def loss2(y_true, y_pred):
    batch_size = tf.cast(tf.shape(y_pred)[0], dtype="int32")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int32")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int32")

    input_length = input_length * tf.ones(shape=(batch_size, 1), dtype="int32")
    label_length = label_length * tf.ones(shape=(batch_size, 1), dtype="int32")
    loss_ = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    
    return loss_
    

In [ ]:
def combined_loss(y_true, y_pred):
    true_provinces_labels,true_char_labels = y_true
    pred_provinces_labels,pred_char_labels = y_pred
    ctc_loss = loss2(true_char_labels,pred_char_labels)
    cls_loss = tf.keras.losses.sparse_categorical_crossentropy(
    true_provinces_labels, pred_provinces_labels, from_logits=False, axis=-1)
    cls_loss = tf.expand_dims(cls_loss,axis=-1)
#     return tf.reduce_sum(ctc_loss,cls_loss)
    return ctc_loss+cls_loss
    

In [ ]:
combined_loss2 = [tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                 loss2]

In [ ]:
combined_loss(labels,rnn_out)

In [ ]:
# loss(labels,rnn_out),loss2(labels,rnn_out)

In [ ]:
model.outputs

In [ ]:
# model.compile(optimizer='adam', loss='mae')

In [ ]:
# model.load_weights('weights/rgb_plate_weights/weights')


In [104]:
opt = tf.keras.optimizers.RMSprop()
model.compile(optimizer=opt, loss=combined_loss2)

In [105]:
callbacks = [
            ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6,verbose=1),
            EarlyStopping(patience=5,restore_best_weights=True,verbose=1),
#             ModelCheckpoint('checkpoints/yolov3_train_{epoch}.tf',
#                             verbose=1, save_weights_only=True),
            tf.keras.callbacks.TerminateOnNaN()
        ]

In [106]:
EPOCHS = 40
STEPS_PER_EPOCH = TRAIN_SIZE//(EPOCHS*BATCH_SIZE)
VAL_STEPS = VAL_SIZE // (EPOCHS*BATCH_SIZE)

In [107]:
STEPS_PER_EPOCH,VAL_STEPS

(404, 78)

In [108]:
history=model.fit(train_ds.repeat(),validation_data=val_ds,epochs=EPOCHS,callbacks=callbacks,steps_per_epoch=STEPS_PER_EPOCH,validation_steps=VAL_STEPS)

Epoch 1/40
404/404 [==============================] - 44s 110ms/step - loss: 20.6041 - output_0_loss: 1.1443 - output_1_loss: 19.4103 - val_loss: 27.4727 - val_output_0_loss: 3.3320 - val_output_1_loss: 24.0649
Epoch 2/40
404/404 [==============================] - 44s 109ms/step - loss: 13.6303 - output_0_loss: 1.0035 - output_1_loss: 12.5141 - val_loss: 10.8878 - val_output_0_loss: 0.9760 - val_output_1_loss: 9.7562
Epoch 3/40
404/404 [==============================] - 43s 107ms/step - loss: 6.8288 - output_0_loss: 0.8522 - output_1_loss: 5.7696 - val_loss: 5.3375 - val_output_0_loss: 0.7790 - val_output_1_loss: 4.3006
Epoch 4/40
404/404 [==============================] - 43s 107ms/step - loss: 4.2738 - output_0_loss: 0.6701 - output_1_loss: 3.2956 - val_loss: 4.4670 - val_output_0_loss: 0.6234 - val_output_1_loss: 3.4891
Epoch 5/40
404/404 [==============================] - 43s 106ms/step - loss: 3.2626 - output_0_loss: 0.5785 - output_1_loss: 2.2889 - val_loss: 3.1678 - val_output_0

In [109]:
model.optimizer.lr

<tf.Variable 'RMSprop/learning_rate:0' shape=() dtype=float32, numpy=4.0000003e-05>

In [16]:
def decode(input):
    batch_size = tf.cast(tf.shape(input)[0], dtype="int32")
    input_length = tf.cast(tf.shape(input)[1], dtype="int32")

    input_length = input_length * tf.ones(shape=(batch_size), dtype="int32")
    decoded = tf.keras.backend.ctc_decode(input,input_length,greedy=True)
#     decoded,_ = tf.nn.ctc_greedy_decoder(
#         tf.transpose(input,[1,0,2]), input_length, merge_repeated=True
#         )
    return decoded

In [111]:
val_img,val_label = next(val_ds.unbatch().batch(3000).as_numpy_iterator())
pred_p,pred_c = model.predict(val_img)
true_p,true_c = val_label
c = 0
for i,l in enumerate(decode(pred_c)[0][0].numpy()[:,:7]):
    if l[-1] == -1:
        if l[:6].tolist() == true_c[i][0:6].tolist():
            c+=1
    else:
        if l.tolist() == true_c[i].tolist():
            c+=1
print("准确率：%4.2f"%(c/3000))

准确率：0.96


In [112]:
# model.save_weights('weights/crnn_plate_weights/weights')

In [113]:
# model.load_weights('weights/crnn_plate_weights/weights')

In [114]:
# model.save('crnn_plate_model')

In [115]:
model.save_weights('weights/rgb_plate_weights/weights')

In [117]:
model.save('rgb_plate_model')

INFO:tensorflow:Assets written to: rgb_plate_model\assets


In [117]:
model.load_weights('weights/rgb_plate_weights/weights')
out = tf.keras.layers.Lambda(lambda x: decode(x))(model.outputs[1])
ctc_model = tf.keras.Model(model.inputs, [model.outputs[0],tf.cast(out[0],tf.int8)], name='ctc_model')

In [118]:
ctc_model.outputs

[<tf.Tensor 'output_0/Softmax:0' shape=(None, 34) dtype=float32>,
 <tf.Tensor 'Cast_4:0' shape=(1, None, None) dtype=int8>]

In [119]:
ctc_model(tf.ones((3,64,224,3)))

[<tf.Tensor: shape=(3, 34), dtype=float32, numpy=
 array([[4.9698660e-06, 2.4301248e-06, 2.6165787e-06, 1.0160279e-06,
         1.4221424e-06, 4.9822988e-06, 6.9477692e-06, 2.0395055e-05,
         3.1504887e-06, 5.2561046e-07, 4.7043604e-06, 9.0076619e-06,
         1.0015608e-06, 7.2509829e-07, 2.6370503e-06, 2.0683804e-06,
         4.6596028e-06, 9.1511702e-06, 9.9988604e-01, 1.8203883e-06,
         1.5120563e-06, 4.9173900e-07, 3.5236585e-06, 4.9727391e-06,
         1.6251294e-06, 5.2749658e-07, 9.7861528e-07, 6.2589729e-06,
         7.0882851e-07, 1.8597227e-06, 3.4194836e-06, 1.7311625e-06,
         1.8513808e-06, 2.7280902e-07],
        [4.9698660e-06, 2.4301248e-06, 2.6165787e-06, 1.0160279e-06,
         1.4221424e-06, 4.9822988e-06, 6.9477692e-06, 2.0395055e-05,
         3.1504887e-06, 5.2561046e-07, 4.7043604e-06, 9.0076619e-06,
         1.0015608e-06, 7.2509829e-07, 2.6370503e-06, 2.0683804e-06,
         4.6596028e-06, 9.1511702e-06, 9.9988604e-01, 1.8203883e-06,
         1.51

In [120]:
converter = tf.lite.TFLiteConverter.from_keras_model(ctc_model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Raymond\AppData\Local\Temp\tmpu9xqk1y7\assets


INFO:tensorflow:Assets written to: C:\Users\Raymond\AppData\Local\Temp\tmpu9xqk1y7\assets


In [121]:
tflite_models_dir = pathlib.Path("tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [122]:
tflite_model_file = tflite_models_dir/"model.tflite"
tflite_model_file.write_bytes(tflite_model)

1762488

In [123]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir/"model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

INFO:tensorflow:Assets written to: C:\Users\Raymond\AppData\Local\Temp\tmpwa1rgao1\assets


INFO:tensorflow:Assets written to: C:\Users\Raymond\AppData\Local\Temp\tmpwa1rgao1\assets


477872

In [122]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()


In [123]:
interpreter.get_input_details()

[{'name': 'input',
  'index': 0,
  'shape': array([  1,  64, 224,   3]),
  'shape_signature': array([ -1,  64, 224,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [124]:
interpreter.get_output_details()

[{'name': 'Identity',
  'index': 133,
  'shape': array([ 1, 34]),
  'shape_signature': array([-1, 34]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'Identity_1',
  'index': 126,
  'shape': array([ 1,  1, 37]),
  'shape_signature': array([-1, -1, 37]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [125]:
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [126]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

In [127]:
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_0_index = interpreter.get_output_details()[0]["index"]
  output_1_index = interpreter.get_output_details()[1]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  prediction_chars = []
  for test_image,test_label in val_ds.unbatch().take(3000).as_numpy_iterator():
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    pred_p = interpreter.get_tensor(output_0_index)
    pred_c = interpreter.get_tensor(output_1_index)
    digit = np.argmax(pred_p[0])
    prediction_digits.append(digit)
    prediction_chars.append(decode(pred_c)[0][0][0].numpy()[:7])
#     tf.print(decode(pred_c())[0][0][0].numpy()[:7])

  # Compare prediction results with ground truth labels to calculate accuracy.
  _,test_labels = next(val_ds.unbatch().batch(3000).as_numpy_iterator())
  accurate_count = 0
#   tf.print(test_labels)
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == test_labels[0][index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)

  c_accurate_count = 0
#   tf.print(test_labels)
  for index in range(len(prediction_chars)):
#     tf.print(prediction_chars[index][:-1],test_labels[1][index][:-1])
    if prediction_chars[index][-1] == -1:
        if prediction_chars[index][:-1].tolist() == test_labels[1][index][:-1].tolist():
          c_accurate_count += 1
    else:
        if prediction_chars[index].tolist() == test_labels[1][index].tolist():
          c_accurate_count += 1
  c_accuracy = c_accurate_count * 1.0 / len(prediction_digits)

  return accuracy,c_accuracy

In [128]:
print(evaluate_model(interpreter))

(0.9676666666666667, 0.9593333333333334)


In [129]:
print(evaluate_model(interpreter_quant))

(0.9666666666666667, 0.9616666666666667)
